In [1]:
# !pip install -q sentence-transformers datasets

In [17]:
from sentence_transformers import SentenceTransformer
bert_model = SentenceTransformer('bert-base-nli-mean-tokens')
bert_model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [18]:
sentences = [
    "the fifty mannequin heads floating in the pool kind of freaked them out",
    "she swore she just saw her sushi move",
    "he embraced his new life as an eggplant",
    "my dentist tells me that chewing bricks is very bad for your teeth",
    "the dental specialist recommended an immediate stop to flossing with construction materials"
]

In [24]:
embs = bert_model.encode(sentences)
embs.shape

(5, 768)

In [25]:
import numpy as np
from sentence_transformers import util
sim = util.cos_sim(embs, embs)
sim

tensor([[1.0000, 0.4091, 0.1091, 0.5007, 0.2994],
        [0.4091, 1.0000, 0.4455, 0.3069, 0.3861],
        [0.1091, 0.4455, 1.0000, 0.2079, 0.2850],
        [0.5007, 0.3069, 0.2079, 1.0000, 0.6385],
        [0.2994, 0.3861, 0.2850, 0.6385, 1.0000]])

In [30]:
for i, emb_vec in enumerate(embs):
    sim = util.cos_sim(emb_vec, embs)
    argmax = sim.sort(descending=True)[1][0][1:][0]
    score =  sim.sort(descending=True)[0][0][1:][0]
    print(score)
    print(sentences[i])
    print(sentences[argmax])
    print('----')

tensor(0.5007)
the fifty mannequin heads floating in the pool kind of freaked them out
my dentist tells me that chewing bricks is very bad for your teeth
----
tensor(0.4455)
she swore she just saw her sushi move
he embraced his new life as an eggplant
----
tensor(0.4455)
he embraced his new life as an eggplant
she swore she just saw her sushi move
----
tensor(0.6385)
my dentist tells me that chewing bricks is very bad for your teeth
the dental specialist recommended an immediate stop to flossing with construction materials
----
tensor(0.6385)
the dental specialist recommended an immediate stop to flossing with construction materials
my dentist tells me that chewing bricks is very bad for your teeth
----


In [31]:
mpnet_model = SentenceTransformer('all-mpnet-base-v2')
embs = mpnet_model.encode(sentences)
embs.shape

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(5, 768)

In [32]:
for i, emb_vec in enumerate(embs):
    sim = util.cos_sim(emb_vec, embs)
    argmax = sim.sort(descending=True)[1][0][1:][0]
    score =  sim.sort(descending=True)[0][0][1:][0]
    print(score)
    print(sentences[i])
    print(sentences[argmax])
    print('----')

tensor(0.2641)
the fifty mannequin heads floating in the pool kind of freaked them out
she swore she just saw her sushi move
----
tensor(0.2641)
she swore she just saw her sushi move
the fifty mannequin heads floating in the pool kind of freaked them out
----
tensor(0.1650)
he embraced his new life as an eggplant
the fifty mannequin heads floating in the pool kind of freaked them out
----
tensor(0.5185)
my dentist tells me that chewing bricks is very bad for your teeth
the dental specialist recommended an immediate stop to flossing with construction materials
----
tensor(0.5185)
the dental specialist recommended an immediate stop to flossing with construction materials
my dentist tells me that chewing bricks is very bad for your teeth
----
